# Maximize and compare the accuracy of the following extractive summarization models: keyBERT, SciBERTSUM, MemSum

Project work to complete the exam in [Text Mining by Prof. Gianluca Moro in the AC 2022/23 in Bologna](https://www.unibo.it/en/teaching/course-unit-catalogue/course-unit/2022/446610).

The dataset is given and already divided into train, test and validation. The models have two targets to predict: relevant sentences and the relevant tokens. The accuracy on both targets should be maximized. The models to be used are:

* [MemSum](https://github.com/nianlonggu/memsum)
* [SciBERTSUM](https://github.com/atharsefid/SciBERTSUM)
* [keyBERT](https://github.com/MaartenGr/KeyBERT)
  * Simply the relevant sentences to be extracted will be those containing at least k relevant tokens, with k = 1, 2 or 3 that is an hyper parameter to be tuned.

The project should be developed in colab with code commented in such a way that every step is understandable even without your verbal explanation in the discussion session.

### Multi-step Episodic Markov decision process extractive SUMmarizer (MemSum)





### SciBERTSUM

### keyBERT

## Preparation

[Recommendations how to install packages](https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/)

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
import numpy as np
import pandas as pd



## Installing the extractive summarization models

### MemSum

## Tuning the hyperparameters of the models

## Comparing the models

## Summary and Conclusion

## Citations

* Nianlong Gu, Elliott Ash, and Richard Hahnloser. 2022. [MemSum: Extractive Summarization of Long Documents Using Multi-Step Episodic Markov Decision Processes.](https://aclanthology.org/2022.acl-long.450) In Proceedings of the 60th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers), pages 6507–6522, Dublin, Ireland. Association for Computational Linguistics.
* Sefid, Athar, and C. Lee Giles. [SciBERTSUM: Extractive Summarization for Scientific Documents.](https://doi.org/10.1007/978-3-031-06555-2_46) Document Analysis Systems: 15th IAPR International Workshop, DAS 2022, La Rochelle, France, May 22–25, 2022, Proceedings. Cham: Springer International Publishing, 2022.
* Grootendorst, Maarten. [KeyBERT: Minimal keyword extraction with BERT.](https://doi.org/10.5281/zenodo.4461265) Zenodo (2020).